# 数据抓取库之urllib
## 我们只需要关心请求的链接是什么，需要传的参数是什么，以及如何设置可选的请求头就好了，不用深入到底层去了解它到底是怎样传输和通信的。

首先，我们看一个例子
urllib库的使用比较简单，下面使用urllib快速爬取一个网页,具体代码如下:
```python
import urllib.request
```
调用urllib.request库的urlopen()方法，并传入一个url
```python
response = urllib.request.urlopen('http://www.baidu.com')
```
使用read()方法读取获取到的网页内容
```python
html = response.read().decode ('utf-8')
```
打印网页内容
```python
print(html)
```



1、首先，了解一下urllib库，它是Python 内置的HTTP请求库，也就是说不需要额外安装即可使用它包含如下4 个模块。
![title](img/urllib_4_module.jpg)

<b>request </b>： 它是最基本的HTTP请求模块，可以用来模拟发送请求。就像在浏览器里输入网址然后回车一样，
只需要给库方法传入URL 以及额外的参数，就可以模拟实现这个过程了。<br>
<b>error </b>： 异常处理模块，如果出现请求错误，我们可以捕获这些异常，然后进行重试或其他操作以保证程序不会意外终止。<br>
<b>parse </b>： 一个工具模块，提供了许多URL 处理方法，比如拆分、解析、合并等。<br>
<b>robotparser </b>：主要是用来识别网站的robots.txt 文件，然后判断哪些网站可以爬，哪些网站不可以爬，它其实用得比较少。<br>
这里重点讲解一下前1个模块。

### 1.1 发送请求
#### urlopen()函数
我们之前只用了近两行代码，便完成了百度搜索的抓取，输出了网页的源代码。得到源代码之后呢？
我们想要的链接、图片地址、文本信息不就都可以提取出来了吗？<br><br>
我们看看它返回的到底是什么。利用type()方法输出响应的类型：
```pytho
import urllib.request
response = urllib.request.urlopen ('http://www.baidu.com')
print(type(response))
```
输出结果如下：
```python
<class 'http.client.HTTPResponse'>
```




通过抓包工具抓取http响应是一个HTTPResposne 类型的对象，主要包含**read()** 、readline()、getheader(name)、
getheaders()、fileno()等方法，以及msg 、version 、status 、reason 、debuglevel 、closed 等属性
![title](img/HTTPResposne.png)


```python
import urllib.request
response = urllib.request.urlopen ('http://www.baidu.com')
print(response.status)                     # 获取响应的状态码
print(response.getheaders())               # 获取响应的头信息
print(response.getheader('Server'))        # 获取服务器类型
print(response.read().decode ('utf-8'))    # 获取响应体内容
```

其实，urllib()方法可以接受多个参数，该方法的定义格式如下：<br>
```python
urllib.request.urlopen(url, data=None, [timeout,]*, cafile = None, capath = None, cadefault = False, context = None)
```
data:参数是可选的，用来指明向服务器发送请求的额外信息。<br>
timeout:参数是可选的，如果请求超出了设置的超时时间，还没有得到响应，就会抛出异常。不指定，则默认。
    
下面用实例来看一下：
```python
import urllib.request
response = urllib.request.urlopen ('https://www.python.org/', timeout = 0.1)
print(response.read())
```
运行结果会出现如下情况：
```python
During handling of the above exception, another exception occurred:

URLError                                  Traceback (most recent call last)
<ipython-input-20-6eaf1c5ecfa4> in <module>()
      1 import urllib.request
----> 2 response = urllib.request.urlopen ('https://www.python.org/', timeout = 0.1)
      3 print(response.read())
    .....
    URLError: <urlopen error [Errno 101] Network is unreachable>
```

### 1.2修改发送请求
#### Request()函数
一些网站为了控制爬虫程序的访问，对HTTP访问请求的Header(头域)中的User-Agent属性进行识别，
如果发现User-Agent并非正常的浏览器时，便会禁止访问。这也造成我们在使用urllib.request.urlopen()函数访间URL时，
经常会遇到返回“403”错误，即禁止访问的情况。这就需要在 urllib.request.urlopen()访问URL之前，修改User-Agent属性，
通过urllib.request模块的Request函数修改。urllib.request.Request()将定义并返回Request对象，在创建Request对象时，
填入headers参数（包含User-Agent信息），注意的是headers参数为字典。如果不添加headers参数，在创建完成之后，
使用add_header()的方法添加。从而伪装成浏览器进行访问。<br>


方法一：实例化Request对象，修改headers参数
```python
from urllib import request
base_url = 'https://tieba.baidu.com/index.html'
headers={}
headers["UserAgent"] = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.84 Safari/537.36'
headers['Referer'] = 'https://tieba.baidu.com/'
req = request.Request(url=base_url,headers=headers)
response = request.urlopen(req)
html = response.read().decode('utf-8')
print(html)
```

方法二：通过Request对象的add_header()方法添加
```python
from urllib import request
base_url = 'https://tieba.baidu.com/index.html'
req = request.Request(base_url)
req.add_header("UserAgent", 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.84 Safari/537.36')
req.add_header("Referer",'https://tieba.baidu.com/')
response = request.urlopen(req)
html = response.read().decode('utf-8')
print(html)
```

### 1.3 下载内容
#### urlretrieve()函数
主要介绍urllib模块提供的urlretrieve()函数。urlretrieve()方法直接将远程数据下载到本地，函数原型如下：<br>
```python
urlretrieve(url, filename=None, reporthook=None, data=None)
```
参数说明：<br>
·参数url是下载文件的超链接。<br>
·参数filename指定了存储的本地路径（如果参数未指定，urllib会生成⼀个临时文件保存数据。）<br>
·参数reporthook是⼀个回调函数。当连接上服务器以及相应的数据块传输完毕时会触发该回调函数，我们可以利用这个回调函数来显示当前的
下载进度。<br>
·参数data指post到服务器的数据，该方法返回⼀个包含两个元素的（filename，headers）元组，filename表示保存到本地的路径，header表示
服务器的响应头。

```python
import urllib,os
urllib.request.urlretrieve('https://www.baidu.com/img/flexible/logo/pc/result.png',os.getcwd()+'/img/baidulogo'+'.jpg')
```

# 数据抓取库之requests
Python爬虫开发中最为常用的方式。Requests实现HTTP请求⾮常简单，操作更加人性化。requests库是第三方模块，需要额外进行安装。
```python
pip install requests
```

request库提供了几乎所有的HTTP请求的方法，如表所示。

| 函数                           | 描述                                                         |
| ------------------------------ | ------------------------------------------------------------ |
| get(url[, timeout = n])        | 对应于HTTP的GET方法，请求指定的网页信息，并返回，timeout设置每次请求超时时间为n秒 |
| head(url)                      | 对应于HTTP的HEAD方法，类似于get请求，获取头信息              |
| post(url,data={'key':'value'}) | 对应于HTTP的post方法，向服务器提交数据，并处理请求，其中字典用于传递客户数据 |
| delete(url)                    | 对应于HTTP的DELETE方法，请求服务器删除指定的页面             |
| options(url)                   | 对应于HTTP的OPTIONS方法，允许客户端查看服务器的性能          |
| put(url,data={'key':'value'})  | 对应于HTTP的PUT方法，从客户端向服务器传送的数据取代指定的文档内容，其中字典用于传递客户数据 |

urllib 库中的urlopen()方法实际上是以GET方式请求网页，而requests中相应的方法就是get()方法，下面通过实例来看一下：
```python
import requests
response = requests.get('https://www.python.org')
type(response)
```
结果如下：
```python
<class 'requests.models.Response'>
```


与浏览器的交互使用一样，get()如同发送HTTP请求，得到一个Response对象，其就是HTTP响应。可以通过Response对象的不同属性来获取不同内容，
使用方法是：对象名.属性名。Response对象的常用属性如表所示。

| 属性及方法         | 描述                                                         |
| ------------------ | ------------------------------------------------------------ |
| text               | HTTP响应内容的字符串形式，即URL对应的页面内容                |
| content            | HTTP响应内容的二进制形式                                     |
| encoding           | HTTP响应内容的编码方式                                       |
| status_code        | HTTP请求的返回状态，为整数，200表示连接成功，404表示连接失败 |
| headers            | 以字典形式返回服务器响应头信息，但是这个字典比较特殊，字典键不区分大小写，若键不存在则返回None |
| json()             | 如果HTTP响应内容包含JSON格式数据，则该方法解析JSON数据。     |
| raise_for_status() | 如果status_code不是200，则产生异常                           |

我们可以通过Response对象的不同属性来获取不同内容
```python
>>>import requests                            #导入requests库
>>>r = requests.get('http://www.baidu.com')   #使用get函数打开百度链接
>>>print(r.text)                              #以字符申形式显示页面内容(有乱码)

<!DOCTYPE html>
<!--STATUS OK--><html> <head><meta http-equiv=content-type content=text/html;
charset=utf-8><meta http-equiv=X-UA-Compatible content=IE=Edge><meta content=always name=referrer>
<link rel=stylesheet type=text/css href=http://s1.bdstatic.com/r/www/cache/bdorz/baidu.min.css>
    <title>ç¾åº¦ä¸ä¸ï¼ä½ å°±ç¥é</title>
......


>>>print(r.content)                          #以二进制形式显示页面内容
b'<!DOCTYPE html>\r\n<!--STATUS OK--><html> <head><meta http-equiv=content-type content=text/html;
;charset=utf-8><meta http-equiv=X-UA-Compatible content=IE=Edge><meta content=always name=referrer>
<link rel=stylesheet type=text/css href=http://s1.bdstatic.com/r/www/cache/bdorz/baidu.min.css>
<title>\xe7\x99\xbe\xe5\xba\xa6\xe4\xb8\x80\xe4\xb8\x8b\xef\xbc\x8c\xe4\xbd\xa0\xe5\xb0\xb1\xe7\x9f\xa5\xe9\x81\x93</title>
......


>>>print(r.encoding)                       #输出页面内容的编码方式
ISO-8859-1
>>>r.encoding ='utf-8'                    #更改编码方式
>>>print(r.text)                          # 再次显示页面内容，己能够正确显示中文
<!DOCTYPE html>
<!--STATUS OK--><html> <head><meta http-equiv=content-type content=text/html;charset=utf-8>
<meta http-equiv=X-UA-Compatible content=IE=Edge><meta content=always name=referrer>
<link rel=stylesheet type=text/css href=http://s1.bdstatic.com/r/www/cache/bdorz/baidu.min.css>
    <title>百度一下，你就知道</title>
    ......

>>>print(r.status_code)                  #愉出连接状态
200
```

除了属性，Response对象还提供了两个常用的方法：<br>
(1)json()方法：如果HTTP响应内容包含JSON格式数据，则该方法解析JSON数据。例如:
```python 
>>>import requests       #导入requests库
>>>r = requests.get("http://httpbin.org/get")encidung
>>>r.json()             #调用json()方法，把返回结果编码成一个json对象

{'headers': {'User-Agent': 'python-requests/2.19.1', 'Accept': '*/*', 'Host': 'httpbin.org',
             'Accept-Encoding': 'gzip, deflate', 'X-Amzn-Trace-Id': 'Root=1-5f2386c8-b7e4ac20f39f6340dccc5d80'},
 'url': 'http://httpbin.org/get', 'origin': '116.85.24.16', 'args': {}}
```
                    
(2)如果 HTTP 请求返回了不成功的状态码， Response.raise_for_status() 会抛出一个 HTTPError异常。                    
```python                    
>>>bad_r = requests.get('http://httpbin.org/status/404')
>>>bad_r.raise_for_status()
                    
   HTTPError: 404 Client Error: NOT FOUND for url: http://httpbin.org/status/404
```

在上面的例子中，我们抓取的是百度的一个页面， 实际上它返回的是一个HTML 文档。如果想抓去图片、音频、视频等文件，应该怎么办呢？
图像等文件本质上都是由二进制码组成的，所以，想要抓取它们，就要拿到它们的二进制码。

```python
>>>import requests
>>>r = requests.get('https://www.baidu.com/img/flexible/logo/pc/result.png')
>>>print(r.text)

�PNG

|Tŵ��w��E!��"Y�J�sRGB����IDATx�]
hE��ӧ�b���U��G���V���H��Y��Y��ZE��l��'
O��O~کY��h�㴑j�B�������j�0e#j�RaP��j�����xn�PN
......

>>>print(r.content)
b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\xca\x00\x00\x00B\x08\x06\x00\x00\x00\x16\x86I\x1d\x00\x00\x00\
x01sRGB\x00\xae\xce\x1c\xe9\x00\x00\x19\x93IDATx\x01\xed]\r|T\xc5\xb5\x9f\xb9w\xbf\xf2E\x0c\x10\x08!\xbb\x81\x00"Y
......

```
接着，我们将刚才提取到的图片保存下来：

```python
import requests
r = requests.get('https://www.baidu.com/img/flexible/logo/pc/result.png')
with open('baidulogo.png','wb') as f:
    f.write(r.content)
```

## 知识回顾

以百度首页为例，使用解析其HTML文档，百度首页及其对应的部分源代码如下：
![title](img/baidulogo.jpg)
![title](img/baidulogo_code.png)

# 数据解析库Beautiful Soup
BeautifulSoup本身并不能访问网页，需要先使用urllib库或者requests库获取网页源代码，然后使用BeautifulSoup解析并提取数据。
beautifulsoup4库中主要的类（模块）是BeautifulSoup，它的实例化对象相当于一个页面。调用BeautifulSoup()函数创建一个BeautifulSoup 对象。
例如：

```python
>>>import urllib.request                  
>>>from bs4 import BeautifulSoup                    # 从bs4 库中导入BeautifulSoup 类
>>>r = urllib.request.urlopen('http://www.baidu.com')  # 使用get 函数打开百度链接
>>>r.encoding = 'utf-8'                             # 更改编码方式
>>> # 下面开始解析，生成解析对象，第一个参数html文档，第二个参数为解析器
>>>soup = BeautifulSoup(r, "html.parser")           # 创建BeautifulSoup 对象
>>>type(soup)                                       # 查看soup 类型
<class 'bs4.BeautifulSoup'>
```



bs4库会将复杂的HTML文档换成树结构，这个结构中的每个节点都是Python对象。
![title](img/html_tree.png)





上述创建的BeautifulSoup 对象得到是一个树形结构，它几乎包含了HTML页面中的标签元素，如head、body等。

就这样BeautifulSoup 对象的某一个属性对应HTML中的标签元素，可通过“对象名.属性名”形式获取属性值。常用的属性名及其含义如表所示。

| 属性   | 描述                                                |
| ------ | --------------------------------------------------- |
| head   | 对应HTML 页面的head内容                         |
| title  | 对应HTML 页面标题，在head中，由title标记    |
| body   | 对应HTML 页面的body内容                         |
| p      | 对应HTML 页面中第一个p内容                      |
| a      | 对应HTML 页面中第一个a内容                      |
| string | 对应HTML 页面所有呈现在Web 上的字符串，即标签的内容 |


通过上图中层次结构来提取数据。


```python
import urllib.request                  
from bs4 import BeautifulSoup                    # 从bs4 库中导入BeautifulSoup 类
r = urllib.request.urlopen('http://www.baidu.com')  # 使用get 函数打开百度链接
r.encoding = 'utf-8'                             # 更改编码方式
soup = BeautifulSoup(r, "html.parser")
# 提取div标签
print(soup.div)
# 提取div下的a标签
print(soup.div.a)         
print(type(soup.div))
print(type(soup.div.a))
```


可以将BeautifulSoup对象理解为对应整个文档的标签树对象，标签树中的具体标签节点也叫Tag对象。Tag
对象有4 个常用属性，如下表：

| 属性    | 描述                                                         |
| ------- | ------------------------------------------------------------ |
| name    | 字符串，标签的名字，如head，title 等                         |
| attrs   | 字典，包含了页面标签的所有属性（尖括号内的其他项），如href   |
| contens | 列表，这个标签下所有子标签的内容                             |
| string  | 字符串，标签所包围的文字，网页中真实的文字（尖括号之间的内容） |

通过Tag对象的属性和方法提取信息
```python
print(soup.a)
print(soup.a.name)
print(soup.a.attrs)
print(soup.a.string)
```

实际上一个网页文件中同一个标签可以出现多次，例如直接调用“soup.a”只能返回第一个标签。当需要列出对应标签的所有内容或找到非第一个标签时，
可以使用BeautifulSoup对象的find_all()方法。该方法会遍历整个HTML文件，按照条件返回标签内容（列表类型）。其语法格式如下：

```python
对象名.find_all(name, attrs, recursive, string, limit)
```

其中参数如下：

name：表示Tag标签名；

attrs：表示按照Tag标签属性值检索（需列出属性名和值）；

recursive表示查找层次（Beautiful Soup默认检索当前标签的所有子孙节点，如果只搜索标签的直接子节点，可以使用参数recursive= False）；

string 表示按照关键字检索string 属性内容（采用string = 开始）；

limit：表示返回结果的个数，默认全部返回。

此外BeautifulSoup类还提供一个find()方法，用于返回找到的第一个结果（字符串），其用法与find_all()方法类似。简单地说，
BeautifulSoup对象的find_all()方法可以根据标签名、标签属性和内容，检索并返回标签列表。还可以通过正则表达式检索片段字符串。

**按属性提取标签**
```python
soup.find('div',attrs={'class':'s-p-top'})
soup.find('img',attrs={'class':'index-logo-src'})
soup.find('img',class_='index-logo-src')
```
这里，把img标签的所有属性都打印出来，得到的类型是一个字典。如果需要单独获取某个属性，可以按以下方式：
```python
print(soup.find('img',class_='index-logo-src')['src'])
# 或者
print(soup.find('img',class_='index-logo-src').get('src'))
```

## 思考：如果使用find_all方法
```python
img = soup.find_all('img',class_='index-logo-src')
type(img),print(img,len(img))
img['src']
```

# 文件管理
## 1、文件的打开和关闭
Python 对文本文件和二进制文件采用统一的操作步骤：

（1）打开文件，或者新建文件；

（2）读/写文件；

（3）关闭文件。

Python系统提供open()函数建立文件对象，并打开要读/写的文件，其语法格式如下：

```python
文件对象名= open(文件名[,打开方式])
```


其中，文件名指定了被打开的文件名称，以字符串形式表示。要打开的文件可以是文本文件或二进制文件。如果文件不在当前工作目录，需要指出文件的路径。文件打开方式，是一个字符串参数，包括只读、写入、追加等，默认是只读（'r'），完整的打开方式如表。

文件打开方式及含义表

| 访问模式（打开方式）| 说明                                                         |
| -------------------- | ------------------------------------------------------------ |
| r                    | 默认模式。以只读方式打开文件。文件的指针将会放在文件的开头。 |
| w                    | 打开一个文件只用于写入。如果该文件已存在则将其覆盖。如果该文件不存在，创建新文件。 |
| a                    | 打开一个文件用于追加。如果该文件已存在，文件指针将会放在文件的结尾。也就是说，新的内容将会被写入到已有内容之后。如果该文件不存在，创建新文件进行写入。 |
| rb                   | 以二进制格式打开一个文件用于只读。文件指针将会放在文件的开头。 |
| wb                   | 以二进制格式打开一个文件只用于写入。如果该文件已存在则将其覆盖。如果该文件不存在，创建新文件。 |
| ab                   | 以二进制格式打开一个文件用于追加。如果该文件已存在，文件指针将会放在文件的结尾。也就是说，新的内容将会被写入到已有内容之后。如果该文件不存在，创建新文件进行写入。 |
| r+                   | 打开一个文件用于读写。文件指针将会放在文件的开头。           |
| w+                   | 打开一个文件用于读写。如果该文件已存在则将其覆盖。如果该文件不存在，创建新文件。 |
| a+                   | 打开一个文件用于读写。如果该文件已存在，文件指针将会放在文件的结尾。文件打开时会是追加模式。如果该文件不存在，创建新文件用于读写。 |
| rb+                  | 以二进制格式打开一个文件用于读写。文件指针将会放在文件的开头。 |
| wb+                  | 以二进制格式打开一个文件用于读写。如果该文件已存在则将其覆盖。如果该文件不存在，创建新文件。 |
| ab+                  | 以二进制格式打开一个文件用于追加。如果该文件已存在，文件指针将会放在文件的结尾。如果该文件不存在，创建新文件用于读写。 |

**文件的关闭**

对于一个已经打开的文件，无论是否进行读写操作，在不需要的情况下应及时关闭，这是中断文件与内存数据存储区的联系，释放打开文件时占用的系统资源。Python提供close()方法关闭文件，假如打开的文件生成一个文件对象file，运行“file.close()”命令即可。例如，以追加方式打开一个名为“test.txt”的文件，然后关闭文件，代码如下：

```python
>>>file = open('test.txt','a')         # 以追加方式打开一个名为“test.txt”的文件
>>>file.close()                       # 关闭这个文件
```



## 2、文件的读写

当向文件中写入数据，Python 提供了2个与文件写入有关的方法：write()方法和writelines()方法。write()方法是向指定文件中写入指定字符串，writelines()方法用于向文件中写入一序列的字符串。这一序列字符串可以是由迭代对象产生的，如一个字符串列表。例如：

```python
>>>file=open ("test.txt","a")
>>>List01 = list('abcd')
>>>file.write(List01)
Traceback (most recent call last):
  File "<pyshell#7>", line 1, in <module>
    file.write(List01)
TypeError: write() argument must be str, not list
>>>file.write(str(List01)+ '\n')             # '\n'表示后续换行后再追加
>>>List02 = [1,2,3,4]
>>>file.writelines(List02)
Traceback (most recent call last):
  File "<pyshell#20>", line 1, in <module>
    file.writelines(List02)
TypeError: write() argument must be str, not int
>>>file.writelines(List01)
>>>file.close()

```


从文件中读取数据时，可以通过多种方式来获取，具体分为以下三种

1. read()方法用于从文件中读取指定的字节数，如果未给定参数或参数为负，则读取整个文件内容，其语法格式如下：

   ```python
   文件对象名.read([size])
   ```


​      其中，size 为从文件中读取的字节数，该方法返回从文件中读取的字符串。

2. 使用readlines方法读取文件

若文件的内容很少，则可以使用readlines方法把整个文件中的内容进行一次性读取。readlines方法会返回一个列表，列表中的每个元素为文件中的每一行数据。

3. 使用readline方法一行一行读数据

使用readline方法可以一行一行地读取文件中的数据。就上述test.txt文件，示例读文件方法，例如：

```python
>>>file = open("test.txt")
>>>content01=file.read()
>>>print(content01)
['a', 'b', 'c', 'd']
abcd
>>>file.close()
>>>file = open("test.txt")
>>>content02=file.readline()
>>>print(content02)
['a', 'b', 'c', 'd']
>>>file.close()
>>>file = open("test.txt")
>>>content03=file.readlines()
>>>print(content03)
["['a', 'b', 'c', 'd']\n", 'abcd']
>>>file.close()
```


## 3、文件的定位读写
### 3.1使用tell方法来获取文件当前的读写位置
在读写文件的过程中，如果想知道当前读取到了文件的哪个位置，则可以使用tell()方法来获取，该方法会返回文件指针的当前位置，例如：

```python
>>>with open('test.txt','r') as file:       # 以只读方式打开原有的名为“test.txt”的文件
    line = file.read(5)                     # 读取前8 个字节
    print(line)                             # 输出前8 个字节
    p = file.tell()                         # 获取指针当前位置
    print('当前位置：',p)                     # 输出当前位置
    line = file.read(4)                     # 继续读取4 个字节
    print(line)                             # 输出读取到的数据
    p = file.tell()                         # 获取指针当前位置
    print('当前位置：',p)                     # 输出当前位置

```

['a',<br>
当前位置： 5<br>
 'b'<br>
当前位置： 9<br>


### 3.2 使用seek方法定位到文件的指定读写位置
如果要指定位置开始读取或者写入文件的数据，则可以使用seek实现，语法如下：

```python
文件对象名.seek(偏移量，起点位置）
```
参数有偏移量（代表需要移动偏移的字节数，正数从前往后偏移，负数从后往前偏移）、起点位置有0，1，2
（表示要从哪个位置开始偏移；0代表从文件开头算起，1代表开始从当前位置开始算起，2代表从文件末尾开始算起，默认为0）
 
           
```python
>>>with open('test.txt','r') as file:
         line = file.read()
         print(line)
         file.seek(5)
         line = file.read()
         print(line)
```           

['a', 'b', 'c', 'd']<br>
abcd<br>
 'b', 'c', 'd']<br>
abcd



## 4、os库
Python 标准库的os 模块除了提供使用操作系统功能和访问文件系统的简便方法之外，还提供了大量文件级操作的方法，下面列举几个常用的方法，如表所示。

| 方法                  | 功能说明                                                     |
| --------------------- | ------------------------------------------------------------ |
| os.rename(src, dst)   | 重命名（从src 到dst）文件或目录，可以实现文件的移动，若目标文件已存在则抛出异常 |
| os.remove(path)       | 删除路径为path 的文件，如果path 是一个文件夹，则抛出异常     |
| os.mkdir(path[,mode]) | 创建目录，要求上级目录必须存在，参数mode 为创建目录的权限，默认创建的目录权限为可读可写可执行 |
| os.getcwd()           | 返回当前工作目录                                             |
| os.chdir(path)        | 将path 设为当前工作目录                                      |
| os.listdir(path)      | 返回path 目录下的文件和目录列表                              |
| os.rmdir(path)        | 删除path 指定的空目录，如果目录非空，则抛出异常              |
| os.removedirs(path)   | 删除多级目录，目录中不能有文件                               |

例如：

```python
>>>import os                            #导入os 模块
>>>os.getcwd()                          #显示当前工作目录
'C:\\Users'
>>>os.mkdir('ostest')                   #创建目录
>>>os.chdir('ostest')                   #将“ostest”目录作为当前目录
>>>os.mkdir('mktest')                   #在“ostest”目录中创建目录“mktest”
>>>f = open('test.txt','w')             #在当前工作目录下创建并打开“test.txt”文件
>>>f.close()                            #关闭文件
>>>os.rename('test.txt','text.txt')     #重命名文件
>>>os.listdir('C:\\Users\\ostest')      #查看文件和目录列表
['text.txt', 'mktest']
>>>os.rmdir('C:\\Users\\ostest\\mktest')      #删除目录
>>>os.listdir('C:\\Users\\ostest')            #再次查看文件和目录列表
['text.txt']
>>>os.remove('C:\\Users\\ostest\\text.txt')   #删除文件

```



os.path模块

os.path模块提供了大量用于路径判断、文件属性获取的方法，os.path常见的方法如表所示。

| 方法                                | 功能说明                                                     |
| ----------------------------------- | ------------------------------------------------------------ |
| os.path.abspath(path)               | 返回给定路径的绝对路径                                       |
| os.path.split(path)                 | 将path分割成目录和文件名二元组返回                           |
| os.path.splitext(path)              | 分离文件名与扩展名：默认返回路径名和文件扩展名的元组         |
| os.path.exists(path)                | 如果路径 path 存在，返回 True；如果路径 path 不存在，返回 False |
| os.path.getsize(path)               | 返回文件大小，如果文件不存在就返回错误                       |
| os.path.join(path1[, path2[, ...]]) | 把目录和文件名合成一个路径，如果各路径名首字母不包含’/’，则函数会自动加上；当有多个路径参数时，则从最后一个以“/”开头的路径参数开始拼接；如果最后一个路径参数为空，则生成的路径以一个’/’分隔符结尾 |
| os.path.isdir(path)                 | 判断路径是否为目录                                           |
| os.path.walk(path, visit, arg)      | 遍历path，进入每个目录都调用visit函数，visit函数必须有3个参数(arg, dirname, names)，dirname表示当前目录的目录名，names代表当前目录下的所有文件名，args则为walk的第三个参数 |

```python
import os
path1 = '/root'
path2 = '/user/'
path3 = 'data'
print(os.path.join(path1,path2,path3))
```

## 5、shutil库

shutil模块也提供了大量方法支持文件和文件夹操作，主要帮助用户复制、移动、改名、备份和删除文件夹等，常用方法如表所示**。**

| 方法                     | 功能说明                                                     |
| ------------------------ | ------------------------------------------------------------ |
| shutil.copy(src,dst)     | 复制文件内容以及权限，如果目标文件已存在则抛出异常           |
| shutil.copy2(src,dst)    | 复制文件内容以及文件的所有状态信息，如果目标文件已存在则抛出异常 |
| shutil.copyfile(src,dst) | 复制文件，不复制文件属性，如果目标文件已存在则直接覆盖       |
| shutil.copytree(src,dst) | 递归复制文件内容及状态信息                                   |
| shutil.rmtree(path)      | 递归删除文件夹                                               |
| shutil.move(src, dst)    | 移动文件或递归移动文件夹，也可给文件和文件夹重命名           |

例如：
```python
import shutil
shutil.move('baidulogo.png','img/')
shutil.copy('img/baidulogo.png','./')
```